by : Karin Dodis

### Used Libraries<a class="anchor" id="chapter1"></a>

In [34]:
import pandas as pd
from sqlalchemy import create_engine
import psycopg2

### Access to the DB <a class="anchor" id="chapter2"></a>

In [35]:
db_config = {'user': 'practicum_student',         # username
             'pwd': 's65BlTKV3faNIGhmvJVzOqhs', # password
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432,              # connection port
             'db': 'data-analyst-fc-barcelona-20-21-db'}          # the name of the database

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                                     db_config['pwd'],
                                                                       db_config['host'],
                                                                       db_config['port'],
                                                                       db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})


In [36]:
engine.table_names()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  """Entry point for launching an IPython kernel.


['squad',
 'games_played',
 'league_stats',
 'champions_stats',
 'cup_stats',
 'supercup_stats']

#  Table Queries <a class="anchor" id="chapter3"></a>

function that takes a query and return dataframe for general use

In [37]:
def queryResult(q):
    return pd.io.sql.read_sql(q, con = engine)

# **1. What is the position played by the majority of the squad?**


In [38]:
query = '''
        SELECT 
          DISTINCT squad.position,
          COUNT(squad.position) as position_count
        FROM
          squad
        GROUP BY
          squad.position
        ORDER BY
          position_count DESC
        '''
queryResult(query)

,position,position_count
0,df,9
1,fw,8
2,mf,8
3,gk,4


# **2. Which players played the most games during the entire season?**



In [39]:
query = '''
        SELECT 
          DISTINCT games_played.player_id,
          squad.first_name,
          squad.last_name,
          SUM(games_played.league) + SUM(games_played.champions)+ SUM(games_played.cup)+ SUM(games_played.supercup) as all_games
        FROM
          squad LEFT JOIN games_played ON games_played.player_id = squad.player_id
        GROUP BY
          games_played.player_id,
          squad.first_name,
          squad.last_name
        ORDER BY
          all_games DESC
        '''
queryResult(query)

,player_id,first_name,last_name,all_games
0,19,Pedro,Lopez,52
1,15,Frenkie,De Jong,51
2,28,Antoine,Griezmann,51
3,14,Sergio,Busquets,50
4,12,Jordi,Alba,49
5,9,Clément,Lenglet,48
6,29,Lionel,Messi,48
7,24,Ousmane,Dembélé,44
8,1,Marc-André,Ter Stegen,42
9,25,Francisco,Trincão,42


# **3. Out of the players who played less than 1000 minutes in the league who scored the most goals?**

In [40]:
query = '''
        SELECT 
          league_stats.player_id,
          squad.first_name,
          squad.last_name,
          league_stats.league_goals
        FROM
          squad LEFT JOIN league_stats ON league_stats.player_id = squad.player_id
        WHERE
          league_stats.league_minutes < 1000
        ORDER BY
          league_stats.league_goals DESC
        '''
queryResult(query)

,player_id,first_name,last_name,league_goals
0,22,Ansu,Fati,4
1,25,Francisco,Trincão,3
2,18,Philippe,Coutinho,2
3,20,Ilaix,Moriba,1
4,13,Sergi,Roberto,1
5,5,Junior,Firpo,1
6,17,Riqui,Puig,1
7,2,Norberto,Murara,0
8,26,Álex,Collado,0
9,3,Iñaki,Peña,0


# **4. Which players played the most minutes in the league? Show top 5**

In [41]:
query = '''
        SELECT 
          league_stats.player_id,
          squad.first_name,
          squad.last_name,
          league_stats.league_minutes
        FROM
          squad LEFT JOIN league_stats ON league_stats.player_id = squad.player_id
        ORDER BY
          league_stats.league_minutes DESC
        LIMIT 5;
        '''
queryResult(query)

,player_id,first_name,last_name,league_minutes
0,15,Frenkie,De Jong,3158
1,12,Jordi,Alba,3030
2,29,Lionel,Messi,3022
3,1,Marc-André,Ter Stegen,2790
4,28,Antoine,Griezmann,2619


# **5.How many assists were made by the forwards?**

In [42]:
query = '''
        SELECT 
          DISTINCT squad.position,
          SUM(league_stats.league_assists) + SUM(champions_stats.champions_assists)+ SUM(cup_stats.cup_assists)+ SUM(supercup_stats.supercup_assists) as assists_sum
        FROM
          squad LEFT JOIN league_stats ON league_stats.player_id = squad.player_id
          LEFT JOIN champions_stats ON champions_stats.player_id = squad.player_id
          LEFT JOIN cup_stats ON cup_stats.player_id = squad.player_id
          LEFT JOIN supercup_stats ON supercup_stats.player_id = squad.player_id
        WHERE
          squad.position = 'fw'
        GROUP BY
          squad.position
        '''
queryResult(query)

,position,assists_sum
0,fw,42


# **6. Who are the top 3 players with the best goals per minute record?**

In [43]:
query = '''
        SELECT 
          DISTINCT league_stats.player_id,
          squad.first_name,
          squad.last_name,
          ((league_stats.league_goals + champions_stats.champions_goals + cup_stats.cup_goals + supercup_stats.supercup_goals) :: real /
          (league_stats.league_minutes  + champions_stats.champions_minutes + cup_stats.cup_minutes + supercup_stats.supercup_minutes) :: real) * 100 as ratio 
        FROM
          squad LEFT JOIN league_stats ON league_stats.player_id = squad.player_id
          LEFT JOIN champions_stats ON champions_stats.player_id = squad.player_id
          LEFT JOIN cup_stats ON cup_stats.player_id = squad.player_id
          LEFT JOIN supercup_stats ON supercup_stats.player_id = squad.player_id
        WHERE
          league_stats.league_goals > 0 OR champions_stats.champions_goals> 0 OR cup_stats.cup_goals > 0 OR supercup_stats.supercup_goals > 0
        ORDER BY
          ratio DESC
        LIMIT 3;
        '''
queryResult(query)

,player_id,first_name,last_name,ratio
0,29,Lionel,Messi,0.906489
1,22,Ansu,Fati,0.838926
2,28,Antoine,Griezmann,0.512426


# **7. How many players did not contribute goals or assists in the league?**

In [44]:
query = '''
        SELECT 
          COUNT(DISTINCT league_stats.player_id) as not_contribute
        FROM
          league_stats
        WHERE
          league_stats.league_goals = 0 AND league_stats.league_assists = 0
 
        '''
queryResult(query)

,not_contribute
0,10


# **8. Who is the player that has the most yellow cards in the league? (If there is a tie then sort by minutes played)**

In [45]:
query = '''
        SELECT 
          league_stats.player_id,
          squad.first_name,
          squad.last_name,
          league_stats.league_yellow
        FROM
          league_stats LEFT JOIN squad ON league_stats.player_id = squad.player_id
        ORDER BY
          league_stats.league_yellow DESC,
          league_stats.league_minutes DESC
        LIMIT 1;
        '''
queryResult(query)

,player_id,first_name,last_name,league_yellow
0,12,Jordi,Alba,9


# **9. Which defender contribute the most goals and assists in the league?**

In [46]:
query = '''
        SELECT 
          league_stats.player_id,
          squad.first_name,
          squad.last_name,
          league_stats.league_goals,
          league_stats.league_assists
        FROM
          league_stats LEFT JOIN squad ON league_stats.player_id = squad.player_id
        WHERE 
          squad.position = 'df'
        ORDER BY
          league_stats.league_goals DESC,
          league_stats.league_assists DESC
        LIMIT 1;
        '''
queryResult(query)

,player_id,first_name,last_name,league_goals,league_assists
0,12,Jordi,Alba,3,7


# **10. How many players played more then 50 hours total in the league and in the champions league?**

In [47]:
query = '''
        SELECT 
          COUNT(DISTINCT league_stats.player_id)

        FROM
          squad LEFT JOIN league_stats ON league_stats.player_id = squad.player_id
          LEFT JOIN champions_stats ON champions_stats.player_id = squad.player_id

        WHERE
          league_stats.league_minutes + champions_stats.champions_minutes > 3000

        '''
queryResult(query)

,count
0,6
